# SPED orientation mapping
[The pyXem SED analysis introduction](https://github.com/pyxem/pyxem-demos/blob/master/Basic%20SED%20Analysis%20-%20GaAs%20Nanowire.ipynb) provides an introduction to [pyXem](https://github.com/pyxem/pyxem). This notebook demonstrates how to run template matching and vector matching to index a SPED dataset. Note that files are assumed to be in the same directory as this notebook. Update as needed.

1. [Load data](#Load-data)
2. [Explore](#Explore)
3. [Preprocessing](#Preprocessing)
4. [Template matching](#Template-matching)
    1. [Build template library](#Build-the-template-library)
    2. [Indexing](#Indexing-with-template-matching)
5. [Vector matching](#Vector-matching)
    1. [Build vector library](#Build-the-vector-library)
    2. [Vector matching: Peak finding](#Vector-matching%3A-Peak-finding)
    2. [Indexing](#Indexing-with-vector-matching)
6. [Results](#Results)

First, load some common dependencies

In [1]:
# You might have tk installed instead of qt
%matplotlib qt
import math
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import pyxem as pxm

import diffpy.structure

from transforms3d.axangles import axangle2mat
from transforms3d.euler import axangle2euler
from transforms3d.euler import euler2mat
from transforms3d.euler import mat2euler

from tqdm import tqdm

## Load data

Load the SPED dataset. The file is lazy-loaded and then cut. This ensures that only required areas are loaded from disk to memory. Here, we load a section of a section of a GaAsSb nanowire, and the parameters in the rest of the notebook are adapted to this dataset. If you explore different datasets, some of these parameters must be updated. The nanowire section contains a transition from a ZB phase to WZ and back to ZB again.

The data type is changed to float and some metadata is set. The call to `pxm.ElectronDiffraction` converts the hyperspy signal to a pyxem object which gives access to the pyxem tools. The metadata from the file has to be copied manually. The constructor probably should have done so automatically, but it does not yet do that.

In [2]:
dp_full = pxm.load('NW_GaAs_ZB_WZ_pyxem_sample.hdf5')

In [ ]:
# The background removal and affine transform (further down) changes
# the type without respecting the loaded precision. We do it ourselves
# to be explicit.
if dp_full.data.dtype != 'float64':
    dp_full.change_dtype('float64')
    
# Reciprocal calibration found by measuring a known interplanar spacing.
# See the pyXem introduction demo for an example
reciprocal_angstrom_per_pixel = 0.032

# Convert to a pyxem ElectronDiffraction, conserve the metadata and add some more
dp_metadata = dp_full.metadata
dp_full = pxm.ElectronDiffraction(dp_full)
dp_full.data *= 1 / dp_full.data.max()
dp_full.metadata = dp_metadata
dp_full.set_diffraction_calibration(reciprocal_angstrom_per_pixel)
dp_full.set_scan_calibration(1.28)

Load structure files using `diffpy`.

In [4]:
structure_zb_file = data_dir + r'\structure_files\GaAs_mp-2534_conventional_standard.cif'
structure_wz_file = data_dir + r'\structure_files\GaAs_mp-8883_conventional_standard.cif'
# structure_zb_file = 'GaAs_mp-2534_conventional_standard.cif'
# structure_wz_file = 'GaAs_mp-8883_conventional_standard.cif'

structure_zb = diffpy.structure.loadStructure(structure_zb_file)
structure_wz = diffpy.structure.loadStructure(structure_wz_file)

## Explore
Before doing anything with the dataset, one way of exploring the dataset is using interactive Virtual Dark Field imaging. First, create a virtual aperture and then pass it to `plot_interactive_virtual_image`.

In [5]:
roi = pxm.roi.CircleROI(cx=0, cy=0, r_inner=0, r=0.07)
dp_full.plot_interactive_virtual_image(roi=roi, cmap='viridis')

In [8]:
crop_region = pxm.roi.RectangularROI(left=90, top=30, right=110, bottom=75)
dp_full.plot(cmap='viridis', vmax=0.8)
crop_region.add_widget(dp_full)

In [9]:
# Crop the dataset for faster execution while reading this notebook. Remove to run on the full dataset
dp = crop_region(dp_full)

## Preprocessing

Preprocessing in this case consist of applying an affine transform to correct for camera distortions and a background removal using a Gaussian difference method. First we look for good parameters for the background removal ($\sigma_{\text{min}}, \sigma_{\text{max}}$).

In [ ]:
dp_test_area = dp.inav[0, 0]  # Try different positions to ensure good values on all parts of the dataset

The following code creates a signal with the test area repeated with the background removed using different parameters. Run it, and find the combination of  ($\sigma_{\text{min}}, \sigma_{\text{max}}$) that removes as much background as possible without affecting the diffraction spots too much. The Difference of Gaussian method works by convolving each diffraction pattern with a Gaussian before subtracting one from the other to create a band-pass filter. $\sigma_{\text{min}}, \sigma_{\text{max}}$ define the standard deviation for the two Gaussians that are applied.

In [ ]:
gauss_stddev_maxs = np.arange(2, 12, 0.2)  # min, max, step
gauss_stddev_mins = np.arange(1,  4, 0.2)  # min, max, step
gauss_processed = np.empty((
    len(gauss_stddev_maxs),
    len(gauss_stddev_mins),
    *dp.axes_manager.signal_shape))

for i, gauss_stddev_max in enumerate(tqdm(gauss_stddev_maxs, leave=False)):
    for j, gauss_stddev_min in enumerate(gauss_stddev_mins):
        gauss_processed[i, j] = dp_test_area.remove_background('gaussian_difference',
                                                          sigma_min=gauss_stddev_min, sigma_max=gauss_stddev_max,
                                                          show_progressbar=False)
dp_gaussian = pxm.ElectronDiffraction(gauss_processed)
dp_gaussian.metadata.General.title = 'Gaussian preprocessed'
dp_gaussian.axes_manager.navigation_axes[0].name = r'$\sigma_{\mathrm{min}}$'
dp_gaussian.axes_manager.navigation_axes[0].offset = gauss_stddev_mins[0]
dp_gaussian.axes_manager.navigation_axes[0].scale = gauss_stddev_mins[1] - gauss_stddev_mins[0]
dp_gaussian.axes_manager.navigation_axes[0].units = ''
dp_gaussian.axes_manager.navigation_axes[1].name = r'$\sigma_{\mathrm{max}}$'
dp_gaussian.axes_manager.navigation_axes[1].offset = gauss_stddev_maxs[0]
dp_gaussian.axes_manager.navigation_axes[1].scale = gauss_stddev_maxs[1] - gauss_stddev_maxs[0]
dp_gaussian.axes_manager.navigation_axes[1].units = ''

In [ ]:
dp_gaussian.plot(cmap='viridis')

I don't have a good solution for finding the camera affine transform parameters, but I have [a notebook](https://github.com/shogas/sped_processing_playground/blob/master/template_param_optimize.ipynb) which runs an optimisation algorithm on diffraction pattern calibration ($Å^{-1}$ per pixel), max excitation error (for relrod length), scale and offset that seems to work OK on my datasets. Suggestions welcome.

Apply the affine transform, the background removal and rescale.

In [ ]:
scale_x = 0.995
scale_y = 1.031
offset_x = 0.631
offset_y = -0.351
dp.apply_affine_transformation(np.array([
    [scale_x, 0, offset_x],
    [0, scale_y, offset_y],
    [0, 0, 1]
    ]))

dp = dp.remove_background('gaussian_difference', sigma_min=2, sigma_max=8)
dp.data -= dp.data.min()
dp.data *= 1 / dp.data.max()

## Template matching
Template matching generates a database of simulated diffraction patterns and then compares all simulated diffraction pattern to each of the experimental diffraction patterns to find the best match.

In [44]:
from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.generators.structure_library_generator import StructureLibraryGenerator
from pyxem.libraries.diffraction_library import load_DiffractionLibrary

### Build the template library

Set parameters and describe the phases present in your sample:

In [45]:
rotation_list_resolution = 1
beam_energy_keV = 200
max_excitation_error = 1/10  # Ångström^{-1}, extent of relrods in reciprocal space. Inverse of specimen thickness is a starting point

phase_descriptions = [('ZB', structure_zb, 'cubic'),
                      ('WZ', structure_wz, 'hexagonal')]
phase_names = [phase[0] for phase in phase_descriptions]
structure_library_generator = StructureLibraryGenerator(phase_descriptions)

inplane_rotations = [[0], [0]]  # The library only needs the base in-plane rotation. The other ones are generated
structure_library = structure_library_generator.get_orientations_from_stereographic_triangle(
        inplane_rotations, rotation_list_resolution)
gen = pxm.DiffractionGenerator(beam_energy_keV, max_excitation_error=max_excitation_error)

target_pattern_dimension_pixels = dp.axes_manager.signal_shape[0]
half_pattern_size = target_pattern_dimension_pixels // 2
reciprocal_radius = reciprocal_angstrom_per_pixel*(half_pattern_size - 1)

If you already have a diffraction library you want to use, load load it from file on disk. Otherwise, create a new one. 

(1) From disk if you already have one:

In [ ]:
diffraction_library_cache_filename = 'GaAs_cubic_hex_1deg.pickle'

In [ ]:
diffraction_library = load_DiffractionLibrary(diffraction_library_cache_filename, safety=True)

(2) Otherwise, generate if from a rotation list on a stereographic triangle:

In [46]:
library_generator = pxm.DiffractionLibraryGenerator(gen)
diffraction_library = library_generator.get_diffraction_library(
    structure_library,
    calibration=reciprocal_angstrom_per_pixel,
    reciprocal_radius=reciprocal_radius,
    half_shape=(half_pattern_size, half_pattern_size),
    with_direct_beam=False)







  0%|                                                                                                                                                                                                               | 0/1052 [00:00<?, ?it/s]





  0%|▍                                                                                                                                                                                                      | 2/1052 [00:00<00:54, 19.43it/s]





  0%|▊                                                                                                                                                                                                      | 4/1052 [00:00<00:54, 19.15it/s]





  1%|█▎                                                                                                                                                                                                     | 7/1052 [00:00<00:52, 19.90it/s]





  1%|█▉               

Optionally, save the library for later use.

In [ ]:
diffraction_library.pickle_library(diffraction_library_cache_filename)

### Indexing with template matching

Given the `diffraction_library` defined above, the `IndexationGenerator` finds the correlation between all patterns in the library and each experimental pattern, and returns the `n_largest` matches with highest correlation.

In [ ]:
indexer = IndexationGenerator(dp, diffraction_library)
indexation_results = indexer.correlate(n_largest=4)

The results are ready for further analysis. The same visualisations can be used for template matching results and vector matching results. An example is given in the [Results](#Results) section below the vector matching to avoid repeating the same code and explanations.

## Vector matching
Another method for generating phase and orientation maps is vector matching. The method is still a work in progress, but it works well for diffraction patterns close to a low-index zone axis.

In [19]:
from pyxem.generators.library_generator import VectorLibraryGenerator
from pyxem.generators.indexation_generator import VectorIndexationGenerator
from pyxem.libraries.structure_library import StructureLibrary
from pyxem.libraries.vector_library import load_VectorLibrary
from pyxem.generators.subpixelrefinement_generator import SubpixelrefinementGenerator
from pyxem.signals.diffraction_vectors import DiffractionVectors

### Build the vector library

In [20]:
reciprocal_radius_max = 2.0  # Extent of library in Å^-1
vector_library_cache_filename = 'GaAs_cubic_hex_vector_2.pickle'

Get a vector library from file on disk or create a new one.

(1) From disk if you already have one:

In [ ]:
vector_library = load_VectorLibrary(vector_library_cache_filename, safety=True)

(2) Generate a new library

In [31]:
# No orientations needed in the structure library nor the vector library generator
structure_library = StructureLibrary(['ZB', 'WZ'], [structure_zb, structure_wz], [[], []])
library_generator = VectorLibraryGenerator(structure_library)
vector_library = library_generator.get_vector_library(reciprocal_radius_max)

Optionally, save the library for later use

In [ ]:
vector_library.pickle_library(vector_library_cache_filename)

###  Vector matching: Peak finding

The first step of vector matching is to find the peaks. Start selecting a method and tuning the parameters interactively.

In [ ]:
dp.find_peaks_interactive(imshow_kwargs={'cmap': 'viridis'}, 'vmax': 0.8

Then run the peak finding on the full dataset with the parameters you found above.

In [ ]:
peaks = dp.find_peaks(method='difference_of_gaussians',
                      min_sigma=0.005,
                      max_sigma=5.0,
                      sigma_ratio=2.0,
                      threshold=0.06,
                      overlap=0.8)

Plot the vectors in a part of the dataset (HyperSpy currently requires a square region).

In [22]:
peaks.inav[0:20, 10:30].plot_diffraction_vectors_on_signal(dp.inav[0:20, 10:30])



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:27<00:00, 98.86it/s]

Remove any peeks that are too long and the direct beam.

In [37]:
def filter_peaks(peaks):
    peaks = peaks[0]
    # Only keep vectors within max length and remove centre (closer than 5px to image centre)
    return peaks[(np.linalg.norm(peaks, axis=1) < reciprocal_radius_max) &
                 (np.any(np.abs(peaks) > 5 * reciprocal_angstrom_per_pixel, axis=1))]

peaks.map(filter_peaks)
# Map changes the signal type. Reset
peaks = DiffractionVectors(peaks.data)
peaks.axes_manager.set_signal_dimension(0)






  0%|                                                                                                                                                                                                                 | 0/90 [00:00<?, ?it/s]




100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 5297.47it/s]

After finding the diffraction spots, the position can be refined using the subpixel refinement generator. The centre of mass method gives good results on the nanowire, but other datasets get better results with another method. See the `SubpixelrefinementGenerator` documentation for other options.

In [38]:
subpixel_refinement = SubpixelrefinementGenerator(dp, peaks)
peaks = DiffractionVectors(subpixel_refinement.center_of_mass_method(square_size=8))
peaks.axes_manager.set_signal_dimension(0)






  0%|                                                                                                                                                                                                                 | 0/90 [00:00<?, ?it/s]




100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 4093.02it/s]




  0%|                                                                                                                                                                                                                 | 0/90 [00:00<?, ?it/s]




 16%|██████████████████████████████▉                                                                                                                                                                        | 14/90 [00:00<00:00, 130.92it/s]




 39%|██████████████████████

Plot the vectors again to see the difference.

In [27]:
peaks.inav[0:20, 15:35].plot_diffraction_vectors_on_signal(dp.inav[0:20, 15:35])

`peaks` now contain the 2D positions of the diffraction spots on the detector. The vector matching method works in 3D coordinates, which are found by projecting the detector positions back onto the Ewald sphere.

In [39]:
beam_energy_keV = 200
camera_length = 0.2  # Not currently used in the calculation, but still a required parameter
peaks.calculate_cartesian_coordinates(beam_energy_keV, camera_length)






  0%|                                                                                                                                                                                                                 | 0/90 [00:00<?, ?it/s]




100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 6926.25it/s]

### Indexing with vector matching

Finally, we are ready to run the indexing. Create an indexation generator and use it to index the vectors

In [40]:
indexation_generator = VectorIndexationGenerator(peaks, vector_library)

In [41]:
indexation_results = indexation_generator.index_vectors(mag_tol=3*reciprocal_angstrom_per_pixel,
                                                angle_tol=4,  # degree
                                                index_error_tol=0.2,
                                                n_peaks_to_index=7,
                                                n_best=2,
                                                show_progressbar=True)






  0%|                                                                                                                                                                                                                 | 0/90 [00:00<?, ?it/s]




  1%|██▏                                                                                                                                                                                                      | 1/90 [00:07<11:04,  7.46s/it]




  2%|████▍                                                                                                                                                                                                    | 2/90 [00:08<08:01,  5.47s/it]




 10%|████████████████████                                                                                                                                                                                     | 9/90 [00:11<05:21,  3.97s/it]




 11%|██████████████████████

The `indexation` results can now be used like the results of template matching. Repeating what we had above:

## Results

Pyxem exposes visualisations for the indexation results through a `CrystallographicMap`. Here, the phase map and orientation maps are plotted along with reliability maps. The orientation maps show the rotation angle in the axis-angle representation of the orientation. [MTEX](https://mtex-toolbox.github.io/) provides better plotting.

In [ ]:
crystal_map = indexation_results.get_crystallographic_map()

In [ ]:
crystal_map.get_phase_map().plot()
crystal_map.get_metric_map('phase_reliability').plot()

In [ ]:
crystal_map.get_orientation_map().plot()
crystal_map.get_metric_map('orientation_reliability').plot()

MTEX gives much better orientation maps, and pyxem supports exporting the orientation data in a format that can be read by mtex.

In [ ]:
crystal_map.save_mtex_map('mtex_orientation_data.csv')

Let's look at the best matches. Due to a `Hyperspy` problem, see 
https://github.com/hyperspy/hyperspy/issues/2080, only a square area can be shown. (Another problem with Hyperspy: The first image shown determines how many markers is shown. When you first open the figure, a ZB position is selected, which has fewer spots. When moving to WZ, some of the spots are missing. The fix for now is to move to WZ, then close and reopen the figure.)

In [47]:
indexation_results.plot_best_matching_results_on_signal(
    dp, diffraction_library, gen, reciprocal_radius)







  0%|                                                                                                                                                                                                                 | 0/90 [00:00<?, ?it/s]





  1%|██▏                                                                                                                                                                                                      | 1/90 [00:00<00:23,  3.82it/s]





  2%|████▍                                                                                                                                                                                                    | 2/90 [00:00<00:20,  4.38it/s]





  9%|█████████████████▊                                                                                                                                                                                       | 8/90 [00:00<00:13,  5.90it/s]





 11%|█████████████████

ValueError: Navigation shape of the marker must be 0 or the same navigation shape as this signal.







100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:20<00:00, 23.94it/s]